In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Loading the dataset:
movie_data = pd.read_csv('E:\CODSOFT\Movie\Movie.csv', encoding='ISO-8859-1')

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_12400\3147604075.py:2: SyntaxWarning: invalid escape sequence '\C'
  movie_data = pd.read_csv('E:\CODSOFT\Movie\Movie.csv', encoding='ISO-8859-1')


In [5]:
# Displaying the basic info and the first few rows of the dataset
print(movie_data.info())
print(movie_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB
None
                                 Name    Year Duration            Genre  \
0                                         NaN      NaN            Drama   
1  #Gadhvi (He thought he was Gandhi)  (2019)  109 min            Drama   
2                         #Homecoming  (2021)   90 min   Drama, Musical   
3                             #Yaaram  (2019)  110 min  Comedy, Ro

# Data Cleaning:

In [7]:
# Convert 'Year' to numeric, extracting only the year if there are any extra characters
movie_data['Year'] = pd.to_numeric(movie_data['Year'].str.extract('(\d{4})')[0], errors='coerce')

# Convert 'Duration' to numeric, extracting only the minutes part
movie_data['Duration'] = pd.to_numeric(movie_data['Duration'].str.replace(' min', ''), errors='coerce')

# Convert 'Votes' to numeric, removing commas if present
movie_data['Votes'] = pd.to_numeric(movie_data['Votes'].str.replace(',', ''), errors='coerce')

# Drop rows with missing 'Rating' values as it’s our target variable
movie_data.dropna(subset=['Rating'], inplace=True)

# Fill remaining missing values or choose a strategy (e.g., fill with 'Unknown' for categorical or median for numerical)
movie_data['Genre'].fillna('Unknown', inplace=True)
movie_data['Director'].fillna('Unknown', inplace=True)
movie_data['Actor 1'].fillna('Unknown', inplace=True)
movie_data['Actor 2'].fillna('Unknown', inplace=True)
movie_data['Actor 3'].fillna('Unknown', inplace=True)

# Inspect the cleaned dataset
print(movie_data.info())


<class 'pandas.core.frame.DataFrame'>
Index: 7919 entries, 1 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      7919 non-null   object 
 1   Year      7919 non-null   float64
 2   Duration  5851 non-null   float64
 3   Genre     7919 non-null   object 
 4   Rating    7919 non-null   float64
 5   Votes     7919 non-null   float64
 6   Director  7919 non-null   object 
 7   Actor 1   7919 non-null   object 
 8   Actor 2   7919 non-null   object 
 9   Actor 3   7919 non-null   object 
dtypes: float64(4), object(6)
memory usage: 680.5+ KB
None


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_12400\2515388563.py:2: SyntaxWarning: invalid escape sequence '\d'
  movie_data['Year'] = pd.to_numeric(movie_data['Year'].str.extract('(\d{4})')[0], errors='coerce')
C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_12400\2515388563.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie_data['Genre'].fillna('Unknown', inplace=True)
C:\Users\Windows 11 Pro\AppData\Local

# Feature Engineering:

In [9]:
# One-hot encode 'Genre' by creating multiple genre columns
movie_data = movie_data.join(movie_data['Genre'].str.get_dummies(sep=', '))

# One-hot encode 'Director' and 'Actor 1' (you may consider using the top 10-20 most common values only)
top_directors = movie_data['Director'].value_counts().nlargest(10).index
movie_data['Director'] = movie_data['Director'].apply(lambda x: x if x in top_directors else 'Other')
movie_data = pd.get_dummies(movie_data, columns=['Director'], drop_first=True)

# Repeat the same for 'Actor 1', 'Actor 2', 'Actor 3' (using top actors to limit columns)
top_actors = movie_data['Actor 1'].value_counts().nlargest(10).index
for actor_col in ['Actor 1', 'Actor 2', 'Actor 3']:
    movie_data[actor_col] = movie_data[actor_col].apply(lambda x: x if x in top_actors else 'Other')
    movie_data = pd.get_dummies(movie_data, columns=[actor_col], drop_first=True)

# Model Building:

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define features (exclude columns like 'Name' that won’t be helpful)
X = movie_data.drop(columns=['Name', 'Rating', 'Genre'])
y = movie_data['Rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model (e.g., RandomForestRegressor)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [14]:
# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 1.2021956459148715
R-squared Score: 0.35336105845970556
